In [43]:
import os
import azureml.core
import pandas as pd
from azureml.core.runconfig import JarLibrary
from azureml.core.compute import ComputeTarget, DatabricksCompute
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.core.databricks import PyPiLibrary

from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.steps import DatabricksStep, PythonScriptStep
from azureml.train.hyperdrive import choice, loguniform

from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


SDK version: 1.47.0


In [44]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


learningmain
learning
eastus
dac8073e-1c2d-4a7d-a53b-c3655e291d58


Add your pip and conda packages to the `RUN pip install` instruction, below.

In [55]:
docker_file = """
FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.3-cudnn8-ubuntu20.04

RUN mkdir -p /usr/share/man/man1 /usr/share/man/man2 && \
    apt-get update &&\
    apt-get install -y --no-install-recommends openjdk-11-jre && \
    apt-get install ca-certificates-java -y && \
    apt-get clean && \
    update-ca-certificates -f;
ENV JAVA_HOME /usr/lib/jvm/java-11-openjdk-amd64/

ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/jdk-11-env

# Install wkhtmltopdf for pdf rendering from html
RUN apt-get -y update && apt-get -y install wkhtmltopdf

# Create conda environment
RUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \
    python=3.8 pip=21.3.1 -c anaconda -c conda-forge

# Prepend path to AzureML conda environment
ENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH

# Install pip dependencies
# markupsafe and itsdangerous are bug workarounds
# RUN pip install 'responsibleai~=0.20.0' \
#                 'mlflow' \
#                 'scikit-learn<1.1'

RUN conda install -p $AZUREML_CONDA_ENVIRONMENT_PATH pytorch torchvision torchaudio pytorch-cuda -c pytorch -c nvidia -y

# This is needed for mpi to locate libpython
ENV LD_LIBRARY_PATH $AZUREML_CONDA_ENVIRONMENT_PATH/lib:$LD_LIBRARY_PATH

"""

In [56]:
print('creating new environment')
jdk_env = Environment("jdk_gpu")
jdk_env.docker.base_image = None
jdk_env.docker.base_dockerfile = docker_file
jdk_env.python.user_managed_dependencies = True
jdk_env = jdk_env.register(ws)
jdk_env.build(ws)

creating new environment


Environment version is set. Attempting to register desired version. To auto-version, reset version to None.


In [48]:
# Set the base image to None, because the image is defined by Dockerfile.

try:
    jdk_env = Environment.get(ws, name="jdk_gpu")
    print(f'environment already exists - version: [{jdk_env.version}]')
except:
    print('creating new environment')
    jdk_env = Environment("jdk_gpu")
    jdk_env.docker.base_image = None
    jdk_env.docker.base_dockerfile = docker_file
    jdk_env.python.user_managed_dependencies = True
    jdk_env = jdk_env.register(ws)
    jdk_env.build(ws)

environment already exists - version: [1]


In [57]:
cluster_name = "gpucluster"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

In [58]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='.',
                      script='test_env.py',
                      compute_target=compute_target,
                      environment=jdk_env)


In [59]:
experiment_name = 'test_java'
experiment = Experiment(workspace=ws, name=experiment_name)




In [60]:
experiment.submit(src)

Experiment,Id,Type,Status,Details Page,Docs Page
test_java,test_java_1675272329_4b3bba9e,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation
